In [4]:
%%javascript
$.getScript('http://120.78.95.32/j_n_contents.js')

<IPython.core.display.Javascript object>

<div id="toc">
</div>

# 4.1

In [5]:
import torch
from torch import nn

In [8]:
class MLP(nn.Module):
    def __init__(self,**kwargs):
        super(MLP,self).__init__(**kwargs)
        self.hidden=nn.Linear(784,256)
        self.act=nn.ReLU()
        self.output=nn.Linear(256,10)
    def forward(self,x):
        a=self.act(self.hidden(x))
        return self.output(a)

In [9]:
x=torch.rand(2,784)
net=MLP()
print(net)
net(x)

MLP(
  (hidden): Linear(in_features=784, out_features=256, bias=True)
  (act): ReLU()
  (output): Linear(in_features=256, out_features=10, bias=True)
)


tensor([[ 0.1299,  0.0159,  0.0638, -0.0603,  0.0092,  0.1026,  0.2844, -0.0253,
          0.0915,  0.0495],
        [-0.0021,  0.0007,  0.2268, -0.0478, -0.0319,  0.0082,  0.2326, -0.0517,
          0.0093, -0.0225]], grad_fn=<AddmmBackward>)

In [10]:
def test(*args):
    for i in range(len(args)):
        print(args[i])

In [13]:
test("yfu",656576,"jyfk")

yfu
656576
jyfk


In [18]:
class mysequential(nn.Module):
    from collections import OrderedDict
    def __init__(self,*args):
        super(mysequential,self).__init__()
        if len(args)==1 and isinstance(args[0],OrderDict):
            for key,module in args[0].items():
                self.add_module(key,module)
        else:
            for idx,module in enumerate(args):
                self.add_module(str(idx),module)
    def forward(self,input):
        for module in self._modules.values():
            input=module(input)
        return input

In [19]:
net=mysequential(
    nn.Linear(784,256),
    nn.ReLU(),
    nn.Linear(256,10)
)
print(net)
net(x)

mysequential(
  (0): Linear(in_features=784, out_features=256, bias=True)
  (1): ReLU()
  (2): Linear(in_features=256, out_features=10, bias=True)
)


tensor([[ 0.0412,  0.1308,  0.1828,  0.0573, -0.0449, -0.2617, -0.1807, -0.1659,
          0.0012, -0.0337],
        [ 0.0439,  0.0725, -0.0023,  0.0685,  0.0180, -0.0558, -0.0221, -0.1046,
          0.1255,  0.0724]], grad_fn=<AddmmBackward>)

In [21]:
net=nn.ModuleDict({
    'linear':nn.Linear(784,256),
    'act':nn.ReLU()
})
net['output']=nn.Linear(256,10)
print(net['linear'])
print(net.output)
print(net)

Linear(in_features=784, out_features=256, bias=True)
Linear(in_features=256, out_features=10, bias=True)
ModuleDict(
  (act): ReLU()
  (linear): Linear(in_features=784, out_features=256, bias=True)
  (output): Linear(in_features=256, out_features=10, bias=True)
)


In [27]:
class fancymlp(nn.Module):
    def __init__(self,**kwargs):
        super(fancymlp,self).__init__(**kwargs)
        self.rand_weight=torch.rand((20,20),requires_grad=False)
        self.linear=nn.Linear(20,20)
        
    def forward(self,x):
        x=self.linear(x)
        x=nn.functional.relu(torch.mm(x,self.rand_weight.data)+1)
        #这里两个全链接层共享参数
        x=self.linear(x)
        while x.norm().item()>1:
            x/=2
        if x.norm().item()<0.8:
            x*=10
        return x.sum()

In [29]:
x=torch.rand(2,20)
net=fancymlp()
print(net)
net(x)

fancymlp(
  (linear): Linear(in_features=20, out_features=20, bias=True)
)


tensor(-2.4397, grad_fn=<SumBackward0>)

In [31]:
class nestmlp(nn.Module):
    def __init__(self,**kwargs):
        super(nestmlp,self).__init__(**kwargs)
        self.net=nn.Sequential(nn.Linear(40,30),nn.ReLU())
    def forward(self,x):
        return self.net(x)
net=nn.Sequential(nestmlp(),nn.Linear(30,20),fancymlp())
x=torch.rand(2,40)
print(net)
net(x)

Sequential(
  (0): nestmlp(
    (net): Sequential(
      (0): Linear(in_features=40, out_features=30, bias=True)
      (1): ReLU()
    )
  )
  (1): Linear(in_features=30, out_features=20, bias=True)
  (2): fancymlp(
    (linear): Linear(in_features=20, out_features=20, bias=True)
  )
)


tensor(-4.3859, grad_fn=<SumBackward0>)

# 4.2

In [32]:
import torch
from torch import nn
from torch.nn import init
net=nn.Sequential(nn.Linear(4,3),nn.ReLU(),nn.Linear(3,1))
print(net)
x=torch.rand(2,4)
y=net(x).sum()

Sequential(
  (0): Linear(in_features=4, out_features=3, bias=True)
  (1): ReLU()
  (2): Linear(in_features=3, out_features=1, bias=True)
)


In [33]:
print(type(net.named_parameters()))

<class 'generator'>


In [39]:
a,b=net.named_parameters().__next__()

In [42]:
a,b

('0.weight', Parameter containing:
 tensor([[ 0.3513,  0.4870,  0.3607,  0.4544],
         [-0.2033, -0.3829, -0.2669,  0.2253],
         [-0.2856,  0.1092,  0.2670,  0.0864]], requires_grad=True))

In [43]:
for n,p in net.named_parameters():
    print(n,p.size())

0.weight torch.Size([3, 4])
0.bias torch.Size([3])
2.weight torch.Size([1, 3])
2.bias torch.Size([1])


In [44]:
for n,p in net[0].named_parameters():
    print(n,p.size(),type(p))

weight torch.Size([3, 4]) <class 'torch.nn.parameter.Parameter'>
bias torch.Size([3]) <class 'torch.nn.parameter.Parameter'>


In [46]:
class mymodel(nn.Module):
    def __init__(self,**kwargs):
        super(mymodel,self).__init__(**kwargs)
        self.weight1=nn.Parameter(torch.rand(20,20))
        self.weight2=torch.rand(20,20)
    def forward(self,x):
        pass
n=mymodel()
for n,p in n.named_parameters():
    print(n)

weight1


In [47]:
weight_0=list(net[0].parameters())[0]
print(weight_0.data)
print(weight_0.grad)

tensor([[ 0.3513,  0.4870,  0.3607,  0.4544],
        [-0.2033, -0.3829, -0.2669,  0.2253],
        [-0.2856,  0.1092,  0.2670,  0.0864]])
None


In [49]:

y.backward()

In [50]:
print(weight_0.grad)

tensor([[0.0827, 0.0830, 0.0597, 0.1073],
        [0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000]])


In [51]:
for n,p in net.named_parameters():
    init.normal_(p,mean=0,std=0.01)
    print(n,p.data)

0.weight tensor([[-0.0010, -0.0134,  0.0020, -0.0034],
        [-0.0066,  0.0014, -0.0066, -0.0131],
        [ 0.0188,  0.0048,  0.0101,  0.0055]])
0.bias tensor([-0.0008, -0.0005, -0.0140])
2.weight tensor([[-0.0072,  0.0009, -0.0060]])
2.bias tensor([-0.0035])


In [53]:
for n,p in net.named_parameters():
    if 'bias' in n:
        init.constant_(p,val=0)
        print(n,p.data)

0.bias tensor([0., 0., 0.])
2.bias tensor([0.])


In [54]:
def normal_(tensor,mean=0,std=1):
    with torch.no_grad():
        return tensor.normal_(mean,std)

In [56]:
def init_weight(tensor):
    with torch.no_grad():
        tensor.uniform_(-10,10)
        tensor*=(tensor.abs()>=5).float()
for n,p in net.named_parameters():
    if 'weight' in n:
        init_weight(p)
        print(n,p.data,p)

0.weight tensor([[-8.0790,  0.0000, -0.0000, -0.0000],
        [ 0.0000,  5.4508, -0.0000,  8.7420],
        [ 0.0000, -0.0000,  0.0000,  0.0000]]) Parameter containing:
tensor([[-8.0790,  0.0000, -0.0000, -0.0000],
        [ 0.0000,  5.4508, -0.0000,  8.7420],
        [ 0.0000, -0.0000,  0.0000,  0.0000]], requires_grad=True)
2.weight tensor([[-0.0000, -6.7450, -0.0000]]) Parameter containing:
tensor([[-0.0000, -6.7450, -0.0000]], requires_grad=True)


In [57]:
linear=nn.Linear(1,1,bias=False)
net=nn.Sequential(linear,linear)
print(net)
for n,p in net.named_parameters():
    init.constant_(p,val=3)
    print(n,p.data)

Sequential(
  (0): Linear(in_features=1, out_features=1, bias=False)
  (1): Linear(in_features=1, out_features=1, bias=False)
)
0.weight tensor([[3.]])


In [58]:
id(net[0])==id(net[1])

True

In [59]:
x=torch.ones(1,1)
y=net(x).sum()
print(y)
y.backward()
print(net[0].weight.grad)

tensor(9., grad_fn=<SumBackward0>)
tensor([[6.]])


# 4.4

In [61]:
import torch
from torch import nn
class centerlayer(nn.Module):
    def __init__(self,**kwargs):
        super(centerlayer,self).__init__(**kwargs)
    def forward(self,x):
        return x-x.mean()

In [62]:
layer=centerlayer()
layer(torch.tensor([1,2,3,4,5],dtype=torch.float))

tensor([-2., -1.,  0.,  1.,  2.])

In [65]:
net=nn.Sequential(nn.Linear(8,128),centerlayer())

In [66]:
y=net(torch.rand(4,8))
y.mean().item()

5.587935447692871e-09

In [76]:
class mydense(nn.Module):
    def __init__(self):
        super(mydense,self).__init__()
        self.params=nn.ParameterList([nn.Parameter(torch.randn(4,4)) for i in range(3)])
        self.params.append(nn.Parameter(torch.randn(4,1)))
    def forward(self,x):
        for i in range(len(self.params)):
            x=torch.mm(x,self.params[i])
        return x
nett=mydense()
print(nett)

mydense(
  (params): ParameterList(
      (0): Parameter containing: [torch.FloatTensor of size 4x4]
      (1): Parameter containing: [torch.FloatTensor of size 4x4]
      (2): Parameter containing: [torch.FloatTensor of size 4x4]
      (3): Parameter containing: [torch.FloatTensor of size 4x1]
  )
)


In [77]:
x=torch.ones(1,4)
print(nett(x))

tensor([[-3.7319]], grad_fn=<MmBackward>)


In [70]:
class mydictdense(nn.Module):
    def __init__(self):
        super(mydictdense,self).__init__()
        self.params=nn.ParameterDict(
        {
            'linear1':nn.Parameter(torch.randn(4,4)),
            'linear2':nn.Parameter(torch.randn(4,1))
        }
        )
        self.params.update({'linear3':nn.Parameter(torch.randn(4,2))})
    def forward(self,x,choice='linear1'):
        return torch.mm(x,self.params[choice])


net=mydictdense()
print(net)

mydictdense(
  (params): ParameterDict(
      (linear1): Parameter containing: [torch.FloatTensor of size 4x4]
      (linear2): Parameter containing: [torch.FloatTensor of size 4x1]
      (linear3): Parameter containing: [torch.FloatTensor of size 4x2]
  )
)


In [71]:
x=torch.ones(1,4)
print(net(x,'linear1'))
print(net(x,'linear2'))
print(net(x,'linear3'))

tensor([[1.0587, 4.7130, 1.1160, 1.9081]], grad_fn=<MmBackward>)
tensor([[2.2800]], grad_fn=<MmBackward>)
tensor([[-2.9252, -0.5998]], grad_fn=<MmBackward>)


In [79]:
net=nn.Sequential(mydictdense(),mydense())

In [80]:
print(net)
print(net(x))

Sequential(
  (0): mydictdense(
    (params): ParameterDict(
        (linear1): Parameter containing: [torch.FloatTensor of size 4x4]
        (linear2): Parameter containing: [torch.FloatTensor of size 4x1]
        (linear3): Parameter containing: [torch.FloatTensor of size 4x2]
    )
  )
  (1): mydense(
    (params): ParameterList(
        (0): Parameter containing: [torch.FloatTensor of size 4x4]
        (1): Parameter containing: [torch.FloatTensor of size 4x4]
        (2): Parameter containing: [torch.FloatTensor of size 4x4]
        (3): Parameter containing: [torch.FloatTensor of size 4x1]
    )
  )
)
tensor([[-1.8665]], grad_fn=<MmBackward>)


# 4.5

In [81]:
import torch
from torch import nn
x=torch.ones(3)
torch.save(x,'x.pt')

In [82]:
x2=torch.load('x.pt')

In [83]:
x2

tensor([1., 1., 1.])

In [85]:
y=torch.zeros(4)
torch.save([x,y],'xy.pt')
xy_list=torch.load('xy.pt')
xy_list

[tensor([1., 1., 1.]), tensor([0., 0., 0., 0.])]

In [86]:
torch.save({'x':x,'y':y},'xy_dict.pt')
xy=torch.load('xy_dict.pt')
xy

{'x': tensor([1., 1., 1.]), 'y': tensor([0., 0., 0., 0.])}

In [88]:
class mlp(nn.Module):
    def __init__(self):
        super(mlp,self).__init__()
        self.hidden=nn.Linear(3,2)
        self.act=nn.ReLU()
        self.output=nn.Linear(2,1)
    def forward(self,x):
        a=self.act(self.hidden(x))
        return self.output(a)
net=mlp()
net.state_dict()

OrderedDict([('hidden.weight', tensor([[-0.4716, -0.5011,  0.3597],
                      [-0.0201,  0.4677,  0.3217]])),
             ('hidden.bias', tensor([-0.4616,  0.4051])),
             ('output.weight', tensor([[-0.0776, -0.1656]])),
             ('output.bias', tensor([-0.0934]))])

In [89]:
optimizer=torch.optim.SGD(net.parameters(),lr=0.001,momentum=0.9)
optimizer.state_dict()

{'state': {},
 'param_groups': [{'lr': 0.001,
   'momentum': 0.9,
   'dampening': 0,
   'weight_decay': 0,
   'nesterov': False,
   'params': [140429638476496,
    140429638478728,
    140429638477864,
    140429638477648]}]}

In [90]:
x=torch.randn(2,3)
y=net(x)
torch.save(net.state_dict(),"net.pt")
net2=mlp()
net2.load_state_dict(torch.load("net.pt"))
y2=net2(x)
y2==y

tensor([[True],
        [True]])

# 4.6

In [91]:
!nvidia-smi

Sun Mar  8 20:02:18 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 418.56       Driver Version: 418.56       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  GeForce GTX 108...  Off  | 00000000:02:00.0 Off |                  N/A |
| 23%   25C    P8     8W / 250W |     10MiB / 11178MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
|   1  GeForce GTX 108...  Off  | 00000000:82:00.0 Off |                  N/A |
| 23%   22C    P8     9W / 250W |     10MiB / 11178MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
        

In [92]:
import torch
from torch import nn
torch.cuda.is_available()

True

In [93]:
torch.cuda.device_count()

2

In [94]:
torch.cuda.current_device()

0

In [96]:
torch.cuda.get_device_name(0)

'GeForce GTX 1080 Ti'

In [100]:
x=torch.tensor([1,2,3])
x=x.to(torch.device("cuda"))

In [101]:
x.device

device(type='cuda', index=0)

In [102]:
net=nn.Linear(3,1)
list(net.parameters())[0].device

device(type='cpu')

In [103]:
net.to(torch.device("cuda"))
list(net.parameters())[0].device

device(type='cuda', index=0)

In [104]:
x=torch.rand(2,3)
x=x.to(torch.device("cuda"))

In [105]:
net(x)

tensor([[0.0129],
        [0.1841]], device='cuda:0', grad_fn=<AddmmBackward>)